In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from refractml import *
from refractml.constants import MLModelFlavours

ModuleNotFoundError: No module named 'category_encoders'

In [2]:
from refractio import snowflake
cars = snowflake.get_dataframe("CAR_NEW100MB")
cars

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


,BUYING_NUM,MAINTENANCE_NUM,DOORS_NUM,PERSONS_NUM,LUGBOOT_NUM,SAFETY_NUM,CLASS_NUM
0,0,3,0,1,0,2,2
1,1,2,0,1,2,2,2
2,2,3,2,2,2,2,0
3,0,1,2,1,2,0,0
4,0,0,2,1,2,1,3
...,...,...,...,...,...,...,...
59849995,3,2,0,0,2,1,2
59849996,1,3,2,2,0,1,3
59849997,0,0,1,2,0,1,0
59849998,0,2,1,1,0,0,0


In [3]:
# cars = pd.read_csv("/data/car_new.csv")
# cars

In [4]:
Y = cars['CLASS_NUM']
X = cars.drop (['CLASS_NUM'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(122688400, 6) (26931600, 6) (122688400,) (26931600,)


In [5]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [6]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [7]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [8]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'BUYING_NUM': {'type': 'integer', 'format': 'int8'}, 'MAINTENANCE_NUM': {'type': 'integer', 'format': 'int8'}, 'DOORS_NUM': {'type': 'integer', 'format': 'int8'}, 'PERSONS_NUM': {'type': 'integer', 'format': 'int8'}, 'LUGBOOT_NUM': {'type': 'integer', 'format': 'int8'}, 'SAFETY_NUM': {'type': 'integer', 'format': 'int8'}}}, 'example': [{'BUYING_NUM': 1, 'MAINTENANCE_NUM': 1, 'DOORS_NUM': 3, 'PERSONS_NUM': 0, 'LUGBOOT_NUM': 1, 'SAFETY_NUM': 2}, {'BUYING_NUM': 3, 'MAINTENANCE_NUM': 3, 'DOORS_NUM': 1, 'PERSONS_NUM': 1, 'LUGBOOT_NUM': 1, 'SAFETY_NUM': 0}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [0]}}


In [10]:
register_model(model, 
               score, 
               name='car_evaluation_new2', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=False, 
               kyd_score = False
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
